In [ ]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1rwQRsjsUgJjFYJH5OhkzOC7PeezZhdEl' -O heart.csv

!pip install scikit-learn==0.22.1

--2021-05-20 13:25:02--  https://docs.google.com/uc?export=download&id=1rwQRsjsUgJjFYJH5OhkzOC7PeezZhdEl
Resolving docs.google.com (docs.google.com)... 142.250.73.206, 2607:f8b0:4004:829::200e
Connecting to docs.google.com (docs.google.com)|142.250.73.206|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-5k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7g9rd1f108m9p4gcue6vju82788ikm95/1621517100000/07828702499945486090/*/1rwQRsjsUgJjFYJH5OhkzOC7PeezZhdEl?e=download [following]
--2021-05-20 13:25:03--  https://doc-0s-5k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7g9rd1f108m9p4gcue6vju82788ikm95/1621517100000/07828702499945486090/*/1rwQRsjsUgJjFYJH5OhkzOC7PeezZhdEl?e=download
Resolving doc-0s-5k-docs.googleusercontent.com (doc-0s-5k-docs.googleusercontent.com)... 142.251.33.193, 2607:f8b0:4004:837::2001
Connecting to doc-0s-5k-docs.googleusercontent.com (doc-0s-5k-docs.googl

In [ ]:
data = pd.read_csv('heart.csv')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=0.25, random_state=13)
X_train.shape, X_test.shape

((227, 13), (76, 13))

In [ ]:
tree = DecisionTreeClassifier(random_state=13, max_depth=None).fit(X_train, y_train)

In [ ]:
predicted_test = tree.predict(X_test)
predicted_train = tree.predict(X_train)

In [ ]:
def get_percent(data1, data2):
    c = 0
    for i in range(len(data1)):
        if data1[i] == data2[i]:
            c += 1
            
    return c / len(data1) * 100

Как видно из следующих цифр, можно наблюдать некий процент переобучения так как, на обучающей выборке процент правильных ответов составляет 100, на тестовой же около 65 процентов.  
Это говорит нам о том, что наше дерево идельно подстроилось под обучающую выборку, но плохо справляется с тестовой.

In [ ]:
print(f"Процент правильных ответов на обучающей выборке = {get_percent(list(predicted_train), list(y_train))}%")

Процент правильных ответов на обучающей выборке = 100.0%


In [ ]:
print(f"Процент правильных ответов на тестовой выборке = {get_percent(list(predicted_test), list(y_test))}%")

Процент правильных ответов на тестовой выборке = 67.10526315789474%


In [ ]:
greeded_tree = DecisionTreeClassifier(random_state=13)

In [ ]:
param_grid = {
    "max_depth": [3, 4, 5, 6, 7, 8, 9, 10, None],
    "max_features": ['auto', 'log2', None],
    "min_samples_leaf": range(1, 10),
    "min_samples_split": range(2, 10),
    "criterion": ['gini', 'entropy']
}

In [ ]:
clf = GridSearchCV(greeded_tree, param_grid, cv=5, scoring='accuracy')

In [ ]:
clf.fit(X_test, y_test)

GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=13,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'criterion': ['gini', 'entropy'],
                     

Теперь посмотрим оптимальные параметры которыем подоюрал Grid Search  
Как видно самым оптимальным значением глубины дерева оказалось значение 5

In [ ]:
clf.best_params_

{'criterion': 'gini',
 'max_depth': 5,
 'max_features': None,
 'min_samples_leaf': 1,
 'min_samples_split': 4}

In [ ]:
# Теперь обучим дерево с полученными параметрами, как видим оно не так сильно подстроилось под обучающие данные, но на тестовых данных процент всё равно низкий
pred_tree = DecisionTreeClassifier(random_state=13, criterion='gini', max_depth=5, max_features=None, min_samples_leaf=1, min_samples_split=4).fit(X_train, y_train)
predicted_test = pred_tree.predict(X_test)
predicted_train = pred_tree.predict(X_train)
print(f"Процент правильных ответов на обучающей выборке = {get_percent(list(predicted_train), list(y_train))}%")
print(f"Процент правильных ответов на тестовой выборке = {get_percent(list(predicted_test), list(y_test))}%")

Процент правильных ответов на обучающей выборке = 94.27312775330397%
Процент правильных ответов на тестовой выборке = 63.1578947368421%
